# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import mplcursors

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sao joao da barra,-21.6403,-41.0511,71.69,86,23,4.23,BR,2023-07-17
1,1,wailua homesteads,22.0669,-159.3780,84.99,70,40,16.11,US,2023-07-17
2,2,port mathurin,-19.6833,63.4167,74.08,79,56,27.96,MU,2023-07-17
3,3,avarua,-21.2078,-159.7750,71.65,88,100,2.30,CK,2023-07-17
4,4,howrah,22.5892,88.3103,80.60,100,40,4.61,IN,2023-07-17


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
# Module 6 - Day 3 - Activity 6
map_plot = city_data_df.hvplot.points(
    x="Lng",
    y="Lat",
    geo=True,
    tiles="OSM",
    frame_width= 700,
    frame_height= 500)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# Drop any rows with null values

ideal = (city_data_df["Max Temp"] >= 65) & (city_data_df["Max Temp"] <= 75)
ideal_df = city_data_df[ideal].dropna()

# Display sample data
ideal_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sao joao da barra,-21.6403,-41.0511,71.69,86,23,4.23,BR,2023-07-17
2,2,port mathurin,-19.6833,63.4167,74.08,79,56,27.96,MU,2023-07-17
3,3,avarua,-21.2078,-159.7750,71.65,88,100,2.30,CK,2023-07-17
6,6,adamstown,-25.0660,-130.1015,70.54,73,18,13.27,PN,2023-07-17
15,15,constantia,44.1833,28.6500,68.68,71,0,9.10,RO,2023-07-17


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
columns= ["City", "Country", "Lat", "Lng", "Humidity"]
hotel_df = hotel_df[columns]
hotel_df['Hotel Name']= ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,sao joao da barra,BR,-21.6403,-41.0511,86,
2,port mathurin,MU,-19.6833,63.4167,79,
3,avarua,CK,-21.2078,-159.7750,88,
6,adamstown,PN,-25.0660,-130.1015,73,
15,constantia,RO,44.1833,28.6500,71,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
# Module 6 - Day 3 - Activity 5
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 30
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng = row["Lng"]
    lat = row["Lat"]
   
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params= params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
sao joao da barra - nearest hotel: Pousada Mediterrâneo
port mathurin - nearest hotel: Escale Vacances
avarua - nearest hotel: Paradise Inn
adamstown - nearest hotel: No hotel found
constantia - nearest hotel: Hotel Chérica
yellowknife - nearest hotel: No hotel found
thompson - nearest hotel: Thompson Inn
terra nova - nearest hotel: No hotel found
kapuskasing - nearest hotel: Travelodge
fort frances - nearest hotel: The Sleepy Owl
knyaze-volkonskoye - nearest hotel: No hotel found
ulety - nearest hotel: No hotel found
along - nearest hotel: Hotel Toshi Palace
west island - nearest hotel: Cocos Village Bungalows
bandar lampung - nearest hotel: Grand Anugerah
booue - nearest hotel: La splendeur de l'équateur
tournus - nearest hotel: Le Relais de l'Abbaye
tadine - nearest hotel: Hôtel Nengoné
slave lake - nearest hotel: Northwest Inn
terrace - nearest hotel: Sandman Hotel Terrace
laojunmiao - nearest hotel: No hotel found
joshimath - nearest hotel: Snow Crest Hotel
n

,City,Country,Lat,Lng,Humidity,Hotel Name
0,sao joao da barra,BR,-21.6403,-41.0511,86,Pousada Mediterrâneo
2,port mathurin,MU,-19.6833,63.4167,79,Escale Vacances
3,avarua,CK,-21.2078,-159.7750,88,Paradise Inn
6,adamstown,PN,-25.0660,-130.1015,73,No hotel found
15,constantia,RO,44.1833,28.6500,71,Hotel Chérica
...,...,...,...,...,...,...
535,babolsar,IR,36.7025,52.6576,76,هتل رزآیند
537,aydarken,KG,39.9443,71.3429,44,No hotel found
543,saintes,FR,45.6667,-0.7500,65,Domaine des Chais
544,ilha de mocambique,MZ,-15.0342,40.7358,71,Ruby Backpackers


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# Module 6 - Day 3 - Activity 6
# https://stackoverflow.com/questions/59678780/show-extra-columns-when-hovering-in-a-scatter-plot-with-hvplot
map_plot_2 = hotel_df.hvplot.points(
    x="Lng",
    y="Lat",
    geo=True,
    tiles="OSM",
    frame_width= 700,
    frame_height= 500,
    hover_cols=["Hotel Name", "Country"]
    )

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Country)